In [ ]:
QUESTION-1

In [ ]:
words = open('names.txt', 'r').read().splitlines()


In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}


import torch


In [ ]:
# create the dataset for trigrams
xs1, xs2, ys = [], [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    print(ch1, ch2, ch3)
    xs1.append(ix1)
    xs2.append(ix2)
    ys.append(ix3)

xs1 = torch.tensor(xs1)
xs2 = torch.tensor(xs2)
ys = torch.tensor(ys)
num = ys.nelement()
print('number of examples: ', num)

In [ ]:
# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True) # Change: The weight matrix now has shape (54, 27)

In [ ]:
import torch.nn.functional as F

In [ ]:
xenc1 = F.one_hot(xs1, num_classes=27).float() # input to the network: one-hot encoding

In [ ]:
xenc2 = F.one_hot(xs2, num_classes=27).float()


In [ ]:
xenc = torch.cat((xenc1, xenc2),1)

In [ ]:
# gradient descent
for k in range(50):

    # forward pass
    xenc1 = F.one_hot(xs1, num_classes=27).float() # one-hot encoding for the first character
    xenc2 = F.one_hot(xs2, num_classes=27).float() # one-hot encoding for the second character
    xenc = torch.cat((xenc1, xenc2), dim=1) # Change: Concatenate the two one-hot encoded vectors
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    print(loss.item())

    # backward pass
    W.grad = None # set to zero the gradient
    loss.backward()

    # update
    W.data += -50.0 * W.grad


In [ ]:
QUESTION-2

In [ ]:
# trigram - create the dataset
xs1, xs2, ys = [], [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        xs1.append(ix1)
        xs2.append(ix2)
        ys.append(ix3)


In [ ]:
# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True) # Change: The weight matrix now has shape (54, 27)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Convert to numpy arrays
xs1 = np.array(xs1)
xs2 = np.array(xs2)
ys = np.array(ys)

# Split the dataset into 80% train, 10% validation, and 10% test
X1_train, X1_temp, X2_train, X2_temp, y_train, y_temp = train_test_split(xs1, xs2, ys, test_size=0.2, random_state=42)
X1_val, X1_test, X2_val, X2_test, y_val, y_test = train_test_split(X1_temp, X2_temp, y_temp, test_size=0.5, random_state=42)

# Convert back to PyTorch tensors
X1_train = torch.tensor(X1_train)
X2_train = torch.tensor(X2_train)
y_train = torch.tensor(y_train)
X1_val = torch.tensor(X1_val)
X2_val = torch.tensor(X2_val)
y_val = torch.tensor(y_val)
X1_test = torch.tensor(X1_test)
X2_test = torch.tensor(X2_test)
y_test = torch.tensor(y_test)


In [ ]:
# gradient descent

num = y_train.nelement()
num_val = y_val.nelement()

for k in range(50):

    # forward pass
    xenc1 = F.one_hot(X1_train, num_classes=27).float() # one-hot encoding for the first character
    xenc2 = F.one_hot(X2_train, num_classes=27).float() # one-hot encoding for the second character
    xenc = torch.cat((xenc1, xenc2), dim=1) # Change: Concatenate the two one-hot encoded vectors
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    loss = -probs[torch.arange(num), y_train].log().mean() + 0.01*(W**2).mean()
    print(f"Epoch {k+1}, Training Loss: {loss.item()}")

    # Forward pass (validation)
    xenc_val1 = F.one_hot(X1_val, num_classes=27).float() # one-hot encoding for the first character
    xenc_val2 = F.one_hot(X2_val, num_classes=27).float() # one-hot encoding for the second character
    xenc_val = torch.cat((xenc_val1, xenc_val2), dim=1) # Change: Concatenate the two one-hot encoded vectors
    logits_val = xenc_val @ W # predict log-counts
    counts_val = logits_val.exp() # counts, equivalent to N
    probs_val = counts_val / counts_val.sum(1, keepdims=True) # probabilities for next character
    loss_val = -probs_val[torch.arange(num_val), y_val].log().mean() + 0.01*(W**2).mean()
    print(f"Epoch {k+1}, Validation Loss: {loss_val.item()}")

    # backward pass
    W.grad = None # set to zero the gradient
    loss.backward()

    # update
    W.data += -50.0 * W.grad

In [ ]:
# We have already split the data into train, dev, and test sets
# The trained model is stored in the variable W

test_num = X1_test.nelement()

# Forward pass (without gradients computation)
with torch.no_grad():
    xenc1_test = F.one_hot(X1_test, num_classes=27).float()
    xenc2_test = F.one_hot(X2_test, num_classes=27).float()
    xenc_test = torch.cat((xenc1_test, xenc1_test), dim=1)
    logits_test = xenc_test @ W # predict log-counts on test data
    counts_test = logits_test.exp() # counts, equivalent to N
    probs_test = counts_test / counts_test.sum(1, keepdims=True) # probabilities for next character
    loss_test = -probs_test[torch.arange(test_num), y_test].log().mean() + 0.01*(W**2).mean() # compute the loss on test data

print("Test Loss:", loss_test.item())


In [ ]:
QUESTION-3

In [ ]:
g = torch.Generator().manual_seed(2147483647)

In [ ]:

regularization_strengths = [0.0000001, 0.000001, 0.00001, 0.00005, 0.0001, 0.001, 0.01, 0.1, 1, 10]
train_losses = []
dev_losses = []

for reg_strength in regularization_strengths:
    # Re-initialize the model with the new regularization strength
    W = torch.randn((54, 27), generator=g, requires_grad=True) # Change: The weight matrix now has shape (54, 27)

    # Train the model on the training set
    for k in range(50):
      # forward pass
      xenc1 = F.one_hot(X1_train, num_classes=27).float()
      xenc2 = F.one_hot(X2_train, num_classes=27).float()
      xenc = torch.cat((xenc1, xenc2), dim=1)
      logits = xenc @ W
      counts = logits.exp()
      probs = counts / counts.sum(1, keepdims=True)
      loss = -probs[torch.arange(num), y_train].log().mean() + reg_strength * (W**2).mean()
      # print(f"Epoch {k+1}, Training Loss: {loss.item()}")

      # backward pass
      W.grad = None
      loss.backward()

      # update
      W.data += -50.0 * W.grad


    # Evaluate the model on the dev set
    xenc_val1 = F.one_hot(X1_val, num_classes=27).float()
    xenc_val2 = F.one_hot(X2_val, num_classes=27).float()
    xenc_val = torch.cat((xenc_val1, xenc_val2), dim=1)
    logits_val = xenc_val @ W
    counts_val = logits_val.exp()
    probs_val = counts_val / counts_val.sum(1, keepdims=True)
    loss_val = -probs_val[torch.arange(num_val), y_val].log().mean() + reg_strength * (W**2).mean()
    print(f"Epoch {k+1}, Validation Loss: {loss_val.item()}")

    # Record the losses
    train_losses.append(loss.item())
    dev_losses.append(loss_val.item())


In [ ]:
# Plot the losses as a function of the regularization strength

import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(regularization_strengths, train_losses, label='Train Loss', marker='o')
plt.plot(regularization_strengths, dev_losses, label='Dev Loss', marker='x')
plt.xlabel('Regularization Strength')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Identify the best regularization strength
best_reg_strength = regularization_strengths[np.argmin(dev_losses)]
best_reg_strength


In [ ]:
# retrain with optimum regularization strength
g = torch.Generator().manual_seed(2147483647 + 1)
W = torch.randn((54, 27), generator=g, requires_grad=True)

In [ ]:
num = y_train.nelement()
num_val = y_val.nelement()

for k in range(50):

    # forward pass
    xenc1 = F.one_hot(X1_train, num_classes=27).float() # one-hot encoding for the first character
    xenc2 = F.one_hot(X2_train, num_classes=27).float() # one-hot encoding for the second character
    xenc = torch.cat((xenc1, xenc2), dim=1) # Change: Concatenate the two one-hot encoded vectors
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    loss = -probs[torch.arange(num), y_train].log().mean() + best_reg_strength*(W**2).mean()
    print(f"Epoch {k+1}, Training Loss: {loss.item()}")

    # Forward pass (validation)
    xenc_val1 = F.one_hot(X1_val, num_classes=27).float() # one-hot encoding for the first character
    xenc_val2 = F.one_hot(X2_val, num_classes=27).float() # one-hot encoding for the second character
    xenc_val = torch.cat((xenc_val1, xenc_val2), dim=1) # Change: Concatenate the two one-hot encoded vectors
    logits_val = xenc_val @ W # predict log-counts
    counts_val = logits_val.exp() # counts, equivalent to N
    probs_val = counts_val / counts_val.sum(1, keepdims=True) # probabilities for next character
    loss_val = -probs_val[torch.arange(num_val), y_val].log().mean() + best_reg_strength*(W**2).mean()
    print(f"Epoch {k+1}, Validation Loss: {loss_val.item()}")

    # backward pass
    W.grad = None # set to zero the gradient
    loss.backward()

    # update
    W.data += -50.0 * W.grad


In [ ]:
# Evaluate on the Test Set:

test_num = X1_test.nelement()

# Forward pass (without gradients computation)
with torch.no_grad():
    xenc1_test = F.one_hot(X1_test, num_classes=27).float()
    xenc2_test = F.one_hot(X2_test, num_classes=27).float()
    xenc_test = torch.cat((xenc1_test, xenc2_test), dim=1) # typo was here!!!!!
    logits_test = xenc_test @ W # predict log-counts on test data
    counts_test = logits_test.exp() # counts, equivalent to N
    probs_test = counts_test / counts_test.sum(1, keepdims=True) # probabilities for next character
    loss_test = -probs_test[torch.arange(test_num), y_test].log().mean() + best_reg_strength*(W**2).mean() # compute the loss on test data

print("Test Loss:", loss_test.item())

In [ ]:
QUESTION-4

In [ ]:
import torch
import torch.nn.functional as F

# Define the small W matrix
W = torch.tensor([[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6],
                  [0.7, 0.8, 0.9]], requires_grad=True)

# Define the input sequence
input_seq = "abc"
stoi = {'a': 0, 'b': 1, 'c': 2}
X1_train = torch.tensor([stoi[c] for c in input_seq])

# One-hot encoding approach
xenc1 = F.one_hot(X1_train, num_classes=3).float()
logits_one_hot = xenc1 @ W
loss_one_hot = -torch.log(logits_one_hot.exp() / logits_one_hot.exp().sum(1, keepdims=True)).mean()
print("Loss using one-hot encoding:", loss_one_hot.item())

# Direct indexing approach
logits_index = W[X1_train]
loss_index = -torch.log(logits_index.exp() / logits_index.exp().sum(1, keepdims=True)).mean()
print("Loss using direct indexing:", loss_index.item())


In [ ]:
import torch
import torch.nn.functional as F

# Define the small W matrix
W = torch.tensor([[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6],
                  [0.7, 0.8, 0.9]], requires_grad=True)

# Define the input sequence
input_seq = "abc"
stoi = {'a': 0, 'b': 1, 'c': 2}
X1_train = torch.tensor([stoi[c] for c in input_seq])
X2_train = torch.roll(X1_train, -1)

# One-hot encoding approach
xenc1 = F.one_hot(X1_train, num_classes=3).float()
xenc2 = F.one_hot(X2_train, num_classes=3).float()
logits_one_hot1 = xenc1 @ W
logits_one_hot2 = xenc2 @ W
loss_one_hot = (-torch.log(logits_one_hot1.exp() / logits_one_hot1.exp().sum(1, keepdims=True)).mean()
                -torch.log(logits_one_hot2.exp() / logits_one_hot2.exp().sum(1, keepdims=True)).mean()) / 2
print("Loss using one-hot encoding:", loss_one_hot.item())

# Direct indexing approach
logits_index1 = W[X1_train]
logits_index2 = W[X2_train]
loss_index = (-torch.log(logits_index1.exp() / logits_index1.exp().sum(1, keepdims=True)).mean()
              -torch.log(logits_index2.exp() / logits_index2.exp().sum(1, keepdims=True)).mean()) / 2
print("Loss using direct indexing:", loss_index.item())


In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}v

In [ ]:
# create the dataset for trigrams
xs1, xs2, ys = [], [], []
for w in words[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    print(ch1, ch2, ch3)
    xs1.append(ix1)
    xs2.append(ix2)
    ys.append(ix3)

xs1 = torch.tensor(xs1)
xs2 = torch.tensor(xs2)
ys = torch.tensor(ys)
num = ys.nelement()
print('number of examples: ', num)

In [ ]:


g = torch.Generator().manual_seed(2147483647 + 1)
W = torch.randn((54, 27), generator=g, requires_grad=True)


In [ ]:
xenc1 = F.one_hot(xs1, num_classes=27).float() # one-hot encoding for the first character

In [ ]:
# much experimentation follows.

xenc1 = F.one_hot(xs1, num_classes=27).float()
xenc2 = F.one_hot(xs2, num_classes=27).float()
xenc = torch.cat((xenc1, xenc2), dim=1)
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
print(loss.item())

In [ ]:
wenc1 = W[xs1]
wenc2 = W[xs2]
wenc = torch.cat((wenc1, wenc2), dim=1)
logits = wenc
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
print(loss.item())


In [ ]:
# One-hot encoding approach
xenc1 = F.one_hot(xs1, num_classes=27).float()
xenc2 = F.one_hot(xs2, num_classes=27).float()
xenc = torch.cat((xenc1, xenc2), dim=1)
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
print(loss.item())

# Direct indexing approach
wenc1 = W[xs1]
wenc2 = W[xs2]
wenc = torch.cat((wenc1, wenc2), dim=1)
logits = wenc  # Use the indexed values directly as logits
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
print(loss.item())


In [ ]:
# One-hot encoding approach
xenc1 = F.one_hot(xs1, num_classes=27).float()
xenc2 = F.one_hot(xs2, num_classes=27).float()
xenc = torch.cat((xenc1, xenc2), dim=1)
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
print(loss.item())

# Direct indexing approach
wenc1 = W[xs1]
wenc2 = W[xs2]
wenc = torch.cat((wenc1, wenc2), dim=1)
logits_d = wenc # Use the indexed values directly as logits
counts_d = logits_d.exp()
probs_d = counts_d / counts_d.sum(1, keepdims=True)
loss_d = -probs_d[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
print(loss_d.item())


In [ ]:
# One-hot encoding approach
xenc1 = F.one_hot(xs1, num_classes=27).float()
xenc2 = F.one_hot(xs2, num_classes=27).float()
logits1 = xenc1 @ W[:27]
logits2 = xenc2 @ W[:27]
logits = torch.cat((logits1, logits2), dim=1)
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
print(loss.item())

# Direct indexing approach
wenc1 = W[xs1]
wenc2 = W[xs2]
wenc = torch.cat((wenc1, wenc2), dim=1)
logits = wenc  # Use the indexed values directly as logits
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
print(loss.item())

In [ ]:
import torch
import torch.nn.functional as F

# Define the small W matrix
W = torch.tensor([[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6],
                  [0.7, 0.8, 0.9]], requires_grad=True)

# Define the input sequence
input_seq = "abc"
stoi = {'a': 0, 'b': 1, 'c': 2}
X1_train = torch.tensor([stoi[c] for c in input_seq])
X2_train = torch.roll(X1_train, -1)

# One-hot encoding approach
xenc1 = F.one_hot(X1_train, num_classes=3).float()
xenc2 = F.one_hot(X2_train, num_classes=3).float()
xenc = torch.cat((xenc1, xenc2), dim=1)
logits_one_hot = xenc @ torch.cat((W, W), dim=0)
loss_one_hot = -torch.log(logits_one_hot.exp() / logits_one_hot.exp().sum(1, keepdims=True)).mean()
print("Loss using one-hot encoding:", loss_one_hot.item())

# Direct indexing approach
wenc1 = W[X1_train]
wenc2 = W[X2_train]
wenc = torch.cat((wenc1, wenc2), dim=1)
logits_index = wenc  # Use the indexed values directly as logits
loss_index = -torch.log(logits_index.exp() / logits_index.exp().sum(1, keepdims=True)).mean()
print("Loss using direct indexing:", loss_index.item())


In [ ]:

import torch
import torch.nn.functional as F

# Define the small W matrix
W = torch.tensor([[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6],
                  [0.7, 0.8, 0.9]], requires_grad=True)

# Define the input sequence
input_seq = "abc"
stoi = {'a': 0, 'b': 1, 'c': 2}
X1_train = torch.tensor([stoi[c] for c in input_seq])
X2_train = torch.roll(X1_train, -1)

# One-hot encoding approach
xenc1 = F.one_hot(X1_train, num_classes=3).float()
xenc2 = F.one_hot(X2_train, num_classes=3).float()
logits_one_hot1 = xenc1 @ W
logits_one_hot2 = xenc2 @ W
loss_one_hot = (-torch.log(logits_one_hot1.exp() / logits_one_hot1.exp().sum(1, keepdims=True)).mean()
                -torch.log(logits_one_hot2.exp() / logits_one_hot2.exp().sum(1, keepdims=True)).mean()) / 2
print("Loss using one-hot encoding:", loss_one_hot.item())

# Direct indexing approach
logits_index1 = W[X1_train]
logits_index2 = W[X2_train]
loss_index = (-torch.log(logits_index1.exp() / logits_index1.exp().sum(1, keepdims=True)).mean()
              -torch.log(logits_index2.exp() / logits_index2.exp().sum(1, keepdims=True)).mean()) / 2
print("Loss using direct indexing:", loss_index.item())


In [ ]:
QUESTION-5

In [ ]:
words = open('names.txt', 'r').read().splitlines()

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
import torch
import torch.nn.functional as F

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True)

# create the dataset for trigrams
xs1, xs2, ys = [], [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs1.append(ix1)
    xs2.append(ix2)
    ys.append(ix3)

xs1 = torch.tensor(xs1)
xs2 = torch.tensor(xs2)
ys = torch.tensor(ys)
num = ys.nelement()
print('number of examples: ', num)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Convert to numpy arrays
xs1 = np.array(xs1)
xs2 = np.array(xs2)
ys = np.array(ys)

# Split the dataset into 80% train, 10% validation, and 10% test
X1_train, X1_temp, X2_train, X2_temp, y_train, y_temp = train_test_split(xs1, xs2, ys, test_size=0.2, random_state=42)
X1_val, X1_test, X2_val, X2_test, y_val, y_test = train_test_split(X1_temp, X2_temp, y_temp, test_size=0.5, random_state=42)

# Convert back to PyTorch tensors
X1_train = torch.tensor(X1_train)
X2_train = torch.tensor(X2_train)
y_train = torch.tensor(y_train)
X1_val = torch.tensor(X1_val)
X2_val = torch.tensor(X2_val)
y_val = torch.tensor(y_val)
X1_test = torch.tensor(X1_test)
X2_test = torch.tensor(X2_test)
y_test = torch.tensor(y_test)



In [ ]:
# gradient descent

best_reg_strength = 0.0001

num = y_train.nelement()
num_val = y_val.nelement()

for k in range(50):

    # forward pass
    xenc1 = F.one_hot(X1_train, num_classes=27).float()
    xenc2 = F.one_hot(X2_train, num_classes=27).float()
    xenc = torch.cat((xenc1, xenc2), dim=1)
    logits = xenc @ W
    # counts = logits.exp()
    # probs = counts / counts.sum(1, keepdims=True)
    # loss = -probs[torch.arange(num), y_train].log().mean() + best_reg_strength*(W**2).mean()
    loss = F.cross_entropy(logits, y_train)
    print(f"Epoch {k+1}, Training Loss: {loss.item()}")

    # Forward pass (validation)
    xenc_val1 = F.one_hot(X1_val, num_classes=27).float()
    xenc_val2 = F.one_hot(X2_val, num_classes=27).float()
    xenc_val = torch.cat((xenc_val1, xenc_val2), dim=1)
    logits_val = xenc_val @ W
    # counts_val = logits_val.exp()
    # probs_val = counts_val / counts_val.sum(1, keepdims=True)
    # loss_val = -probs_val[torch.arange(num_val), y_val].log().mean() + best_reg_strength*(W**2).mean()
    loss_val = F.cross_entropy(logits_val, y_val) + best_reg_strength*(W**2).mean()
    print(f"Epoch {k+1}, Validation Loss: {loss_val.item()}")

    # backward pass
    W.grad = None # set to zero the gradient
    loss.backward()

    # update
    W.data += -50.0 * W.grad


    test_num = X1_test.nelement()

In [ ]:
# Forward pass (without gradients computation)
with torch.no_grad():
    xenc1_test = F.one_hot(X1_test, num_classes=27).float()
    xenc2_test = F.one_hot(X2_test, num_classes=27).float()
    xenc_test = torch.cat((xenc1_test, xenc2_test), dim=1)
    logits_test = xenc_test @ W
    loss_test = F.cross_entropy(logits_test, y_test) + best_reg_strength*(W**2).mean()

print("Test Loss:", loss_test.item())


In [ ]:
The difference in the loss values observed between the NLL and cross-entropy implementations is likely due to numerical stability improvements in the cross-entropy computation. In machine learning, numerical stability refers to the ability of an algorithm to produce accurate and consistent results even when dealing with very small or very large numbers. This is important because computers have finite precision, and small numerical errors can accumulate over time, leading to inaccurate results.

In the NLL implementation, you manually computed the exponential of the logits and then normalized them to get probabilities. This approach can be numerically unstable when dealing with very large or very small logits, as the exponential function can quickly overflow or underflow.

On the other hand, the cross-entropy loss in PyTorch combines the softmax activation and the NLL computation into one step. This function is implemented in a way that is more numerically stable, as it avoids the explicit computation of the exponential and normalization steps. Instead, it directly computes the log probabilities in a way that is less prone to numerical issues.

The difference in loss values you observed is likely due to this improved numerical stability in the cross-entropy computation. It is expected to see small differences in the loss values when switching between the two methods. However, the difference you observed is not significant, and it is unlikely to have a meaningful impact on the model's performance.

In conclusion, using the cross-entropy loss in PyTorch is a good choice, as it provides a more numerically stable and efficient way to compute the loss compared to manually computing the NLL.